In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import *
from sklearn import svm
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import *
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import *
from sklearn.decomposition import *
from sklearn.impute import *
from sklearn.pipeline import Pipeline
from sklearn.datasets import *
from sklearn.feature_selection import *
import numpy as np
import random
import time
import csv


In [5]:
## Importante sempre especificar a seed para garantir a reproducilidade dos resultados
## No caso do sklearn tem que especificar o random_state
RANDOM_STATE = 42

In [6]:
df = pd.read_csv("arrhythmia.csv", header=None).replace("?", np.nan)
data = df.to_numpy()
x, y = data[:, :-1], data[:, -1]
y = pd.Series(y).apply(str)
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,test_size=0.2,stratify=y, random_state=RANDOM_STATE)

#Teste pra corrigir aviso de UserWarning: Features [ 19  21  47  67  69  83  85 131 132 139 141 143 144 145 151 156 157 164 204 264 274] are constant.
selector = VarianceThreshold(threshold=0.01)
x_train = selector.fit_transform(x_train)
x_test = selector.transform(x_test)

In [4]:
def criar_individuo_decisionTree():
    
    start_time = time.time()
    # Parametros do StandardScaler
    with_mean = random.choice([True, False])
    with_std = random.choice([True, False])
    
    # Parametros do SimpleImputer
    imputer_strategies = ["mean", "median", "most_frequent", "constant"]
    imputer_strategy = random.choice(imputer_strategies)
    
    if imputer_strategy == "constant":
        fill_value = random.uniform(0.0, 1.0)
    else:
        fill_value = None
    
    # Parametros do SelectKBest
    k_best_params = {
        "k": random.randint(10, 50)
    }
    
    # Parametros do classificador (por exemplo, DecisionTreeClassifier)
    clf_params = {
        "max_depth": random.randint(1, 10),
        "min_samples_split": random.randint(2, 20), 
        "random_state": RANDOM_STATE
    }
    
    individuo = [
        ('scaler', StandardScaler(with_mean=with_mean, with_std=with_std)),
        ('imputer', SimpleImputer(strategy=imputer_strategy, fill_value=fill_value)),
        ('feature-selection', SelectKBest(**k_best_params)),
        ('classifier', DecisionTreeClassifier(**clf_params))
    ]
    
    individuo_dict = {
        'scaler_params': {'with_mean': with_mean, 'with_std': with_std},
        'imputer_params': {'strategy': imputer_strategy, 'fill_value': fill_value},
        'feature_selection_params': k_best_params,
        'classifier_params': clf_params
    }
    
    return individuo, start_time, individuo_dict

In [5]:
#scaler_params = {"with_mean":with_mean, "with_std":with_std}
#imputer_params = {"strategy": imputer_strategy, "fill_value": fill_value}
#feature_selection_params = {"k": 31}
#clf_params = {"max_depth": 10, "min_samples_split": 2, "random_state": RANDOM_STATE}

column_names = ['Individuo', 'Tempo de execucao', 'Accuracy', 'F1 Score']

for _ in range(10):
    individuo, start_time, individuo_dict = criar_individuo_decisionTree()
    pipeline = Pipeline(individuo)
    pipeline.fit(x_train, y_train)
    
    #predições
    predictions = pipeline.predict(x_test)
    
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=1.0)

    accuracy = pipeline.score(x_test, y_test)
    report = classification_report(y_test, predictions)
    
    end_time = time.time()
    tempo = end_time - start_time

    with open('resultados_decisionTree.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        
        if file.tell() == 0:
            writer.writerow(column_names)
        writer.writerow([individuo_dict, tempo, accuracy, f1])

C:\Users\maria\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\maria\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\maria\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\maria\anaconda3\Lib\site-packages\sklearn\metr

In [9]:
def criar_individuo_randomForest():
    
    start_time = time.time()
    
    params_a = [True, True, 'mean', None, 18, 17, 4, RANDOM_STATE]

    def evaluate():
        
        with_mean, with_std, strategy, fill_value, n_estimators, max_depth, min_samples_split, random_state = params_a
        
        imputer = SimpleImputer(strategy=strategy)
        selector = SelectKBest(k=k)

        x_train_imputed = imputer.fit_transform(x_train)
        x_test_imputed = imputer.transform(x_test)
        
        x_train_selected = selector.fit_transform(x_train_imputed, y_train)
        x_test_selected = selector.fit_transform(x_test_imputed, y_test)
        
        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            random_state=random_state
        )
        model.fit(x_train_selected, y_train)
        y_pred = model.predict(x_test_selected)
        f1 = f1_score(y_test, y_pred, average='weighted')
        #print("F1-score do evaluate:", f1)
        return f1,

    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()
    toolbox.register("attr_int", np.random.randint, 1, 100)
    toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.attr_int, ), n=4)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    toolbox.register("evaluate", evaluate)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutUniformInt, low=1, up=100, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)

    population = toolbox.population(n=10)

    algorithms.eaMuPlusLambda(population, toolbox, mu=10, lambda_=50, cxpb=0.7, mutpb=0.3, ngen=10, stats=None, halloffame=None)

    best_individual = tools.selBest(population, k=1)[0]
    print("Melhores hiperparâmetros encontrados:", best_individual)
                                               
    n_estimators, max_depth, min_samples_split, random_state = best_individual
    best_model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        random_state=random_state
    )

    best_model.fit(x_train_selected, y_train)
    y_pred = best_model.predict(x_test_selected)

    f1 = f1_score(y_test, y_pred, average='weighted')
    print("F1-score:", f1)
    
    return best_individual, start_time, f1

In [10]:
column_names = [' Melhor Individuo', 'Tempo de execucao', 'F1 Score']

for _ in range(10):
    best_individual, start_time, f1 = criar_individuo_randomForest()
    
    end_time = time.time()
    tempo = end_time - start_time

    with open('resultados_randomForest.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        
        if file.tell() == 0:
            writer.writerow(column_names)
        writer.writerow([best_individual, tempo, f1])

##scaler_params = {"with_mean":True, "with_std":True}
##imputer_params = {"strategy": "most_frequent"}
##feature_selection_params = {"k": 36}
##clf_params = {"n_estimators": 10, "max_depth": 13, "min_samples_split": 2, "random_state": RANDOM_STATE}

TypeError: criar_individuo_randomForest.<locals>.evaluate() takes 0 positional arguments but 1 was given

In [8]:
def criar_individuo_SVC():
    
    start_time = time.time()
    # Parametros do StandardScaler
    with_mean = random.choice([True, False])
    with_std = random.choice([True, False])
    
    # Parametros do SimpleImputer
    imputer_strategies = ["mean", "median", "most_frequent", "constant"]
    imputer_strategy = random.choice(imputer_strategies)
    
    if imputer_strategy == "constant":
        fill_value = random.uniform(0.0, 1.0)
    else:
        fill_value = None
    
    # Parametros do SelectKBest
    k_best_params = {
        "k": random.randint(10, 50)
    }
    
    clf_params = {
        "C": random.uniform(0.1, 2.0),
        "kernel": random.choice(["linear", "poly", "rbf", "sigmoid"]),
        "gamma": random.choice(["scale", "auto"]),
        "tol": 10**(-random.randint(3, 7)), 
        "random_state": RANDOM_STATE
    }
    
    individuo = [
        ('scaler', StandardScaler(with_mean=with_mean, with_std=with_std)),
        ('imputer', SimpleImputer(strategy=imputer_strategy, fill_value=fill_value)),
        ('feature-selection', SelectKBest(**k_best_params)),
        ('classifier', svm.SVC(**clf_params))
    ]
    
    individuo_dict = {
        'scaler_params': {'with_mean': with_mean, 'with_std': with_std},
        'imputer_params': {'strategy': imputer_strategy, 'fill_value': fill_value},
        'feature_selection_params': k_best_params,
        'classifier_params': clf_params
    }
    
    return individuo, start_time, individuo_dict

In [9]:
##scaler_params = {"with_mean":True, "with_std":True}
##imputer_params = {"strategy": "median"}
##feature_selection_params = {"k": 37}
##clf_params = {"C": 1, "kernel": "sigmoid", "gamma": "auto", "tol": 10E-5, "random_state": RANDOM_STATE}

for _ in range(10):
    individuo, start_time, individuo_dict = criar_individuo_SVC()
    pipeline = Pipeline(individuo)
    pipeline.fit(x_train, y_train)
    
    #predições
    predictions = pipeline.predict(x_test)
    
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=1.0)

    accuracy = pipeline.score(x_test, y_test)
    report = classification_report(y_test, predictions)
    
    end_time = time.time()
    tempo = end_time - start_time

    with open('resultados_SVC.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        
        if file.tell() == 0:
            writer.writerow(column_names)
        writer.writerow([individuo_dict, tempo, accuracy, f1])

C:\Users\maria\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\maria\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\maria\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\maria\anaconda3\Lib\site-packages\sklearn\metr

In [10]:
def criar_individuo_AdaBoost():
    
    start_time = time.time()
    # Parametros do StandardScaler
    with_mean = random.choice([True, False])
    with_std = random.choice([True, False])
    
    # Parametros do SimpleImputer
    imputer_strategies = ["mean", "median", "most_frequent", "constant"]
    imputer_strategy = random.choice(imputer_strategies)
    
    if imputer_strategy == "constant":
        fill_value = random.uniform(0.0, 1.0)
    else:
        fill_value = None
    
    # Parametros do SelectKBest
    k_best_params = {
        "k": random.randint(10, 50)
    }
    
    clf_params = {
        "n_estimators": random.randint(1, 100),
        "random_state": RANDOM_STATE
    }
    
    individuo = [
        ('scaler', StandardScaler(with_mean=with_mean, with_std=with_std)),
        ('imputer', SimpleImputer(strategy=imputer_strategy, fill_value=fill_value)),
        ('feature-selection', SelectKBest(**k_best_params)),
        ('classifier', AdaBoostClassifier(**clf_params))
    ]
    
    individuo_dict = {
        'scaler_params': {'with_mean': with_mean, 'with_std': with_std},
        'imputer_params': {'strategy': imputer_strategy, 'fill_value': fill_value},
        'feature_selection_params': k_best_params,
        'classifier_params': clf_params
    }
    
    return individuo, start_time, individuo_dict

In [11]:
for _ in range(100):
    individuo, start_time, individuo_dict = criar_individuo_AdaBoost()
    pipeline = Pipeline(individuo)
    pipeline.fit(x_train, y_train)
    
    #predições
    predictions = pipeline.predict(x_test)
    
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=1.0)

    accuracy = pipeline.score(x_test, y_test)
    report = classification_report(y_test, predictions)
    
    end_time = time.time()
    tempo = end_time - start_time

    with open('resultados_AdaBoost.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        
        if file.tell() == 0:
            writer.writerow(column_names)
        writer.writerow([individuo_dict, tempo, accuracy, f1])

C:\Users\maria\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\maria\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\maria\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\maria\anaconda3\Lib\site-packages\sklearn\metr

In [12]:
pip install deap


Note: you may need to restart the kernel to use updated packages.


In [8]:
from deap import base, creator, tools, algorithms
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [56]:
scaler_params = {"with_mean": True, "with_std": True}
imputer_params = {"strategy": 'mean', "fill_value": None}
feature_selection_params = {"k": 10}
clf_params = {'n_estimators': 18, "max_depth": 17, "min_samples_split": 4, "random_state": RANDOM_STATE}

#with_mean = True
#with_std = True
#strategy = 'mean'
#fill_value = None
#k = 10
#n_estimators = 18
#max_depth = 17
#min_samples_split = 4



In [62]:


for _ in range(10):
    individuo, start_time, individuo_dict = criar_individuo_randomForest()
    pipeline = Pipeline(individuo)
    pipeline.fit(x_train, y_train)
    
    #predições
    predictions = pipeline.predict(x_test)
    
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=1.0)

    accuracy = pipeline.score(x_test, y_test)
    report = classification_report(y_test, predictions)
    
    end_time = time.time()
    tempo = end_time - start_time

    with open('resultados_randomForest.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        
        if file.tell() == 0:
            writer.writerow(column_names)
        writer.writerow([individuo_dict, tempo, accuracy, f1])

gen	nevals
0  	10    
1  	50    
2  	50    
3  	50    
4  	50    
5  	50    
6  	50    
7  	50    
8  	50    
9  	50    
10 	50    
Melhores hiperparâmetros encontrados: [68, 2, 73, 66]
best_params: [68, 2, 73, 66]
F1-score do modelo com os melhores hiperparâmetros no conjunto de teste: 0.453893609764055
